In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import cvxpy as cvx
from PIL import Image
from copy import deepcopy

Create a 25 by 5 matrix R and a matrix M that is the product of R and R transpose. M should have rank at most 5.

In [27]:
R = np.random.randn(125).reshape((25, 5))
print R.shape

(25L, 5L)


In [28]:
M = np.dot(R, R.T)
print M.shape

(25L, 25L)


In [4]:
import random

Create a matrix Mhat that has a probability of 0.60 that entries are replaced by 0. The goal is to recover M while only knowing Mhat.

In [29]:
Mhat60 = deepcopy(M)
for i in xrange(25):
    for j in xrange(25):
        if random.random() < 0.60:
            Mhat60[i, j] = 0

In [30]:
print np.mean(Mhat60 == M)

0.3872


Nuclear norm minimization uses convex optimization to solve this problem.  Want to minimize the rank of a matrix U subject to the constraint that U matches the known entries of Mhat. This can be restated as minimizing the L0 norm of the eigenvalues of U subject to the constraints that U is symmetric and matches the known entries of Mhat. Unfortunately this problem is NP-hard but under certain restrictions, minimizing the L1 norm returns the same solution. This can be formulated as minimizing the trace of matrix U subject to the constraints that U is symmetric and positive semi-definite and also matches the known entries of Mhat. The code below formulates this problem using the CVXPY package.

In [31]:
known = np.zeros((25, 25))
for i in xrange(25):
    for j in xrange(25):
        if Mhat60[i, j] != 0:
            known[i, j] = 1
U = cvx.Semidef(25)
obj = cvx.Minimize(cvx.trace(U))
constraints = [cvx.mul_elemwise(known, U) == cvx.mul_elemwise(known, Mhat60), U == U.T]
prob = cvx.Problem(obj, constraints)
prob.solve(verbose = True, solver=cvx.CVXOPT)

     pcost       dcost       gap    pres   dres   k/t
 0:  5.7611e+01  5.7611e+01  5e+02  6e+01  1e+00  1e+00
 1:  1.2610e+02  1.2949e+02  1e+02  1e+01  2e-01  4e+00
 2:  1.2445e+02  1.2515e+02  1e+01  2e+00  4e-02  7e-01
 3:  1.2391e+02  1.2404e+02  3e+00  4e-01  7e-03  1e-01
 4:  1.2377e+02  1.2379e+02  4e-01  6e-02  1e-03  2e-02
 5:  1.2377e+02  1.2378e+02  2e-01  3e-02  5e-04  9e-03
 6:  1.2376e+02  1.2376e+02  4e-02  6e-03  1e-04  2e-03
 7:  1.2376e+02  1.2376e+02  1e-02  1e-03  2e-05  4e-04
 8:  1.2376e+02  1.2376e+02  2e-03  3e-04  5e-06  1e-04
 9:  1.2376e+02  1.2376e+02  6e-04  8e-05  1e-06  2e-05
10:  1.2376e+02  1.2376e+02  1e-04  2e-05  3e-07  5e-06
11:  1.2376e+02  1.2376e+02  2e-05  3e-06  5e-08  1e-06
12:  1.2376e+02  1.2376e+02  5e-06  6e-07  1e-08  2e-07
13:  1.2376e+02  1.2376e+02  8e-07  1e-07  2e-09  3e-08
14:  1.2376e+02  1.2376e+02  1e-07  2e-08  3e-10  6e-09
Optimal solution found.


123.75592269378912

When roughly 60% of M's entries are missing, this method is able to find an exact solution.

In [32]:
print "Frobenius norm using nuclear norm minimization:", np.linalg.norm(U.value - M)

Frobenius norm using nuclear norm minimization: 5.97625861756e-08


Now use SVD to solve the same problem, this time for matrices where roughly 80% of entries are missing. SVD performs relatively poorly on this problem.

In [9]:
trials = 20
for t in xrange(trials):
    R = np.random.randn(125).reshape((25, 5))
    M = np.dot(R, R.T)
    Mhat80 = deepcopy(M)
    for i in xrange(25):
        for j in xrange(25):
            if random.random() < 0.80:
                Mhat80[i, j] = 0
    U, sigma, V = np.linalg.svd(Mhat80)
    recover = np.matrix(U[:, :5]) * np.diag(sigma[:5]) * np.matrix(V[:5, :])
    print "Trial %d: %0.2f" %(t, np.linalg.norm(recover - M))

Trial 0: 65.17
Trial 1: 56.33
Trial 2: 62.75
Trial 3: 58.79
Trial 4: 56.82
Trial 5: 49.98
Trial 6: 55.01
Trial 7: 55.76
Trial 8: 47.15
Trial 9: 56.51
Trial 10: 47.46
Trial 11: 51.25
Trial 12: 49.66
Trial 13: 56.02
Trial 14: 83.64
Trial 15: 45.21
Trial 16: 54.65
Trial 17: 61.10
Trial 18: 60.84
Trial 19: 65.07


When nuclear norm minimization is used with 80% of entries missing, recovering the exact M not longer occurs but this method still performs significantly better than using SVD.

In [34]:
for t in xrange(1, 21, 1):
    R = np.random.randn(125).reshape((25, 5))
    M = np.dot(R, R.T)
    Mhat80 = deepcopy(M)
    for i in xrange(25):
        for j in xrange(25):
            if random.random() < 0.80:
                Mhat80[i, j] = 0
    known = np.zeros((25, 25))
    for i in xrange(25):
        for j in xrange(25):
            if Mhat80[i, j] != 0:
                known[i, j] = 1
    U = cvx.Semidef(25)
    obj = cvx.Minimize(cvx.trace(U))
    constraints = [cvx.mul_elemwise(known, U) == cvx.mul_elemwise(known, Mhat80), U == U.T]
    prob = cvx.Problem(obj, constraints)
    prob.solve(verbose = False, solver=cvx.CVXOPT)
    print "Trial %d: %0.2f" %(t, np.linalg.norm(U.value - M))

Trial 1: 17.85
Trial 2: 31.45
Trial 3: 25.99
Trial 4: 29.79
Trial 5: 14.53
Trial 6: 29.60
Trial 7: 23.62
Trial 8: 28.86
Trial 9: 26.55
Trial 10: 25.57
Trial 11: 23.72
Trial 12: 27.54
Trial 13: 26.23
Trial 14: 24.93
Trial 15: 17.64
Trial 16: 27.38
Trial 17: 22.87
Trial 18: 30.49
Trial 19: 25.31
Trial 20: 19.46


In [42]:
def multiTrial(trials=21, missing=0.70):
    print "Nuclear norm matrix completion with %d percent entries missing." %(missing*100)
    for t in xrange(1, trials, 1):
        R = np.random.randn(125).reshape((25, 5))
        M = np.dot(R, R.T)
        Mhat = deepcopy(M)
        for i in xrange(25):
            for j in xrange(25):
                if random.random() < missing:
                    Mhat[i, j] = 0
        known = np.zeros((25, 25))
        for i in xrange(25):
            for j in xrange(25):
                if Mhat[i, j] != 0:
                    known[i, j] = 1
        U = cvx.Semidef(25)
        obj = cvx.Minimize(cvx.trace(U))
        constraints = [cvx.mul_elemwise(known, U) == cvx.mul_elemwise(known, Mhat), U == U.T]
        prob = cvx.Problem(obj, constraints)
        prob.solve(verbose = False, solver=cvx.CVXOPT)
        print "Trial %d: Percent Missing: %0.2f Norm: %0.2f" %(t, 1.0 - np.mean(known), np.linalg.norm(U.value - M))

Even with 70% probability of entries missing, nuclear norm minimization is sometimes still able to extract an exact recovery of M from Mhat.

In [43]:
multiTrial(missing=0.70, trials=26)

Nuclear norm matrix completion with 70 percent entries missing.
Trial 1: Percent Missing: 0.67 Norm: 1.26
Trial 2: Percent Missing: 0.72 Norm: 7.99
Trial 3: Percent Missing: 0.68 Norm: 1.65
Trial 4: Percent Missing: 0.70 Norm: 0.00
Trial 5: Percent Missing: 0.67 Norm: 5.20
Trial 6: Percent Missing: 0.70 Norm: 2.75
Trial 7: Percent Missing: 0.74 Norm: 14.41
Trial 8: Percent Missing: 0.67 Norm: 0.00
Trial 9: Percent Missing: 0.71 Norm: 12.96
Trial 10: Percent Missing: 0.68 Norm: 7.23
Trial 11: Percent Missing: 0.72 Norm: 9.02
Trial 12: Percent Missing: 0.67 Norm: 1.17
Trial 13: Percent Missing: 0.67 Norm: 0.17
Trial 14: Percent Missing: 0.70 Norm: 5.85
Trial 15: Percent Missing: 0.68 Norm: 8.84
Trial 16: Percent Missing: 0.70 Norm: 0.00
Trial 17: Percent Missing: 0.68 Norm: 6.71
Trial 18: Percent Missing: 0.70 Norm: 0.33
Trial 19: Percent Missing: 0.70 Norm: 12.16
Trial 20: Percent Missing: 0.72 Norm: 3.75
Trial 21: Percent Missing: 0.71 Norm: 9.37
Trial 22: Percent Missing: 0.70 Norm: 3